## Dataset Creation 

This takes a long time to run in Jupyter server (roughly 70 minutes). Will work on making this more efficient. The week02_variable_dataset.ipynb file will pull from the saved output created here.

In [1]:
# Imports 
import time
import requests
import pandas as pd
#%pip install yfinance --quiet
import yfinance as yf
from pathlib import Path
import numpy as np
import warnings
import zipfile
from pathlib import Path

warnings.filterwarnings("ignore")

In [2]:
import os
repo_url = "https://github.com/tongyuguo/HelpHerInvest.git"
repo_dir = "HelpHerInvest"
if not os.path.exists(repo_dir):
    !git clone {repo_url}
else:
    !git -C {repo_dir} pull

# list available data files
!ls -lah {repo_dir}/Data

error: Pulling is not possible because you have unmerged files.
hint: Fix them up in the work tree, and then use 'git add/rm <file>'
hint: as appropriate to mark resolution and make a commit.
fatal: Exiting because of an unresolved conflict.
total 13M
drwxr-xr-x  2 jupyter-toomeyck jupyter-toomeyck  132 Jan 30 22:17 .
drwxr-xr-x 11 jupyter-toomeyck jupyter-toomeyck 4.0K Jan 31 00:24 ..
-rw-r--r--  1 jupyter-toomeyck jupyter-toomeyck 2.4M Jan 31 00:26 dependent_variables.csv
-rw-r--r--  1 jupyter-toomeyck jupyter-toomeyck 673K Jan 30 22:35 independent_variables.csv
-rw-r--r--  1 jupyter-toomeyck jupyter-toomeyck 9.4M Jan 30 23:23 stock_symbols_new.csv.zip
-rw-r--r--  1 jupyter-toomeyck jupyter-toomeyck 296K Jan 30 21:45 testing_small.csv.zip


In [3]:

## PARAMETERS ##
## CHANGE OUTPUT PATH ##
# X-Variables #

DATA_DIR = Path(repo_dir) / "Data"
## Create Path to Dataset ##
#repo_root = Path("/home/jupyter-toomeyck/HelpHerInvest")
output_path = DATA_DIR / "stock_symbols_new.csv.zip"
output_path.parent.mkdir(parents=True, exist_ok=True)
output_file = "stock_symbols_new.csv.zip"

SEC_URL = "https://www.sec.gov/files/company_tickers_exchange.json"
SEC_HEADERS = {"User-Agent": "YourAppName your_email@example.com"}  # required by SEC


def get_universe_from_sec(limit):
    r = requests.get(SEC_URL, headers=SEC_HEADERS, timeout=30)
    r.raise_for_status()
    j = r.json()
    df = pd.DataFrame(j["data"], columns=j["fields"])
    df = df.rename(columns={"ticker": "symbol", "name": "company_name"})
    df["symbol"] = df["symbol"].str.upper()
    return df[["symbol", "company_name"]].drop_duplicates()

def yf_fetch_info(symbol: str) -> dict:
    # Normalize common Yahoo symbol formatting
    candidates = [symbol, symbol.replace("-", ".")]
    for sym in candidates:
        try:
            t = yf.Ticker(sym)
            info = t.get_info()  # yfinance >= 0.2.0 style
            if info and isinstance(info, dict) and info.get("quoteType") in ("EQUITY", "ETF"):
                return info
        except Exception:
            pass

    return info

def build_base_table(limit, sleep_s=0.35):
    universe = get_universe_from_sec(limit=limit)
    print("Symbols pulled:",len(universe.index))
    rows = []
    count = 0
    for sym in universe["symbol"].tolist():
        rows.append(yf_fetch_info(sym))
        count += 1
        time.sleep(sleep_s)  # throttle to avoid Yahoo blocks
        if count % 200 == 0:
            print("{} rows completed".format(count))

    facts = pd.DataFrame(rows)

    df_cols = list(facts.columns)
    added = ["symbol", "company_name"]
    cols = df_cols + added

    base = (
        universe
        .merge(facts, on="symbol", how="left")
        [cols]
        .drop_duplicates(subset=["symbol"])
    )

    return base

df_base = build_base_table(limit=None, sleep_s=0.35)


print(df_base.head(10))
print(df_base.shape)
print(df_base.columns)

## CHANGE THE OUTPUT PATH ##

# Save as CSV inside a zip
df_base.to_csv(output_path, index=False, compression="zip")
print(f"Saved: {output_path}")

'''
df_base.to_csv(output_path,index=False)
df = pd.read_csv(output_path)
grouped = df.groupby("sector")["sector"].count()
print(grouped)
'''

Symbols pulled: 10338
200 rows completed
400 rows completed
600 rows completed
800 rows completed
1000 rows completed
1200 rows completed
1400 rows completed
1600 rows completed


HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: WNS"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: WNS"}}}


1800 rows completed
2000 rows completed
2200 rows completed
2400 rows completed
2600 rows completed


HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: TIXT"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: TIXT"}}}


2800 rows completed
3000 rows completed
3200 rows completed
3400 rows completed
3600 rows completed
3800 rows completed
4000 rows completed
4200 rows completed
4400 rows completed
4600 rows completed


...

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: KPHMW"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: KPHMW"}}}


9000 rows completed
9200 rows completed


HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: SKYH-WT"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: SKYH.WT"}}}


9400 rows completed
9600 rows completed
9800 rows completed
10000 rows completed
10200 rows completed
                    address1           city state         zip        country  \
0  2788 San Tomas Expressway    Santa Clara    CA       95051  United States   
1  1600 Amphitheatre Parkway  Mountain View    CA       94043  United States   
2         One Apple Park Way      Cupertino    CA       95014  United States   
3          One Microsoft Way        Redmond    WA  98052-6399  United States   
4     410 Terry Avenue North        Seattle    WA  98109-5210  United States   
5                 1 Meta Way     Menlo Park    CA       94025  United States   
6          3421 Hillview Ave      Palo Alto    CA       94304  United States   
7               1 Tesla Road         Austin    TX       78725  United States   
8         3555 Farnam Street          Omaha    NE       68131  United States   
9           1 Customer Drive    Bentonville    AR       72716  United States   

            phone

'\ndf_base.to_csv(output_path,index=False)\ndf = pd.read_csv(output_path)\ngrouped = df.groupby("sector")["sector"].count()\nprint(grouped)\n'